In [ ]:
### Imports
import pypsa
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import sys
import importlib

# Verbindungen zu den Ordnern der Einzelaufgaben

sys.path.append("../processed_spatial_res")
from get_spatial_res import get_spatial_res

sys.path.append("../processed_capacity_factors")
import cutout_local
importlib.reload(cutout_local)
from cutout_local import(get_cutout,get_availability,get_pv_and_wind)       #TODO: get_cutout Error -> C:\\Users\steli muss remote im repo oder remote abgelegt werden


sys.path.append("../processed_existing_conv_pp")
import get_existing_conv_pp
importlib.reload(get_existing_conv_pp)
from get_existing_conv_pp import get_existing_conv_pp

sys.path.append("../processed_load_profile")
import load_processing
importlib.reload(load_processing)
from load_processing import regional_load

sys.path.append("../processed_RE_potential")
import get_RE_potential
importlib.reload(get_RE_potential)
from get_RE_potential import get_RE_potential

In [3]:
### Spatial resolution

regions = get_spatial_res()
regions.head()


,geometry,GID_0,COUNTRY,NAME_1,REP_POINT
region_5,,,,,
Kärnten/Steiermark,"POLYGON ((14.43537 46.44343, 14.43537 46.44345...",AUT,Austria,Kärnten,POINT (14.98609 47.10053)
Niederösterreich/Burgenland,"POLYGON ((16.06715 46.84889, 16.0672 46.84884,...",AUT,Austria,Burgenland,POINT (15.92138 47.92607)
Oberösterreich/Salzburg,"POLYGON ((13.69947 47.00479, 13.69858 47.00534...",AUT,Austria,Oberösterreich,POINT (13.86111 47.85873)
Tirol/Vorarlberg,"MULTIPOLYGON (((12.39612 46.71143, 12.39598 46...",AUT,Austria,Tirol,POINT (10.82959 47.25349)
Wien,"POLYGON ((16.29905 48.12999, 16.29897 48.12999...",AUT,Austria,Wien,POINT (16.37248 48.2208)


In [4]:
### Load Zeitreihe

load_regions = regional_load()
load_regions.head()
print(load_regions)

                     Kärnten/Steiermark  Niederösterreich/Burgenland  \
time                                                                   
2013-01-01 00:00:00         6713.397737                  7392.725220   
2013-01-01 01:00:00         6785.509837                  7472.134330   
2013-01-01 02:00:00         6790.853449                  7478.018661   
2013-01-01 03:00:00         6902.301714                  7600.744356   
2013-01-01 04:00:00         6902.301714                  7600.744356   
...                                 ...                          ...   
2013-12-31 19:00:00         5285.800593                  5820.669783   
2013-12-31 20:00:00         5714.984417                  6293.282639   
2013-12-31 21:00:00         6141.433730                  6762.884280   
2013-12-31 22:00:00         6671.314808                  7346.383927   
2013-12-31 23:00:00         6685.125026                  7361.591599   

                     Oberösterreich/Salzburg  Tirol/Vorarlberg 

In [ ]:
### Weitere Imports der Teilaufgaben

cutout = get_cutout()
ex_wind, ex_solar,_shape = get_RE_potential()
A_pv, A_w = get_availability(cutout=cutout, regions=regions, excluder_solar=ex_solar, excluder_wind=ex_wind)
pv_cap, pv_cf, wind_cap, wind_cf = get_pv_and_wind(cutout=cutout, regions=regions, A_pv=A_pv, A_w=A_w)

#Capacity 
pv_cf_df   = pv_cf.to_pandas()
wind_cf_df = wind_cf.to_pandas()
pv_cap_s   = pv_cap.to_pandas()
wind_cap_s = wind_cap.to_pandas()


TypeError: Arguments 'time' and 'module' must be specified. Spatial bounds must either be passed via argument 'bounds' or 'x' and 'y'.

In [ ]:
### Removes 2020 timestep to only have 2019 time series 

weather_year = pv_cf_df.index[0].year  
pv_cf_df   = pv_cf_df[pv_cf_df.index.year == weather_year]
wind_cf_df = wind_cf_df[wind_cf_df.index.year == weather_year]


In [ ]:
load_mapped = load_regions.copy()
load_mapped.index = pd.to_datetime(load_mapped.index).map(lambda t: t.replace(year=weather_year))
print(load_mapped)

                     Kärnten/Steiermark  Niederösterreich/Burgenland  \
time                                                                   
2019-01-01 00:00:00         6713.397737                  7392.725220   
2019-01-01 01:00:00         6785.509837                  7472.134330   
2019-01-01 02:00:00         6790.853449                  7478.018661   
2019-01-01 03:00:00         6902.301714                  7600.744356   
2019-01-01 04:00:00         6902.301714                  7600.744356   
...                                 ...                          ...   
2019-12-31 19:00:00         5285.800593                  5820.669783   
2019-12-31 20:00:00         5714.984417                  6293.282639   
2019-12-31 21:00:00         6141.433730                  6762.884280   
2019-12-31 22:00:00         6671.314808                  7346.383927   
2019-12-31 23:00:00         6685.125026                  7361.591599   

                     Oberösterreich/Salzburg  Tirol/Vorarlberg 

In [ ]:
TIME_RES = "3h" 

if TIME_RES is not None:
    load_mapped = load_mapped.resample(TIME_RES).mean()
    pv_cf_df    = pv_cf_df.resample(TIME_RES).mean()
    wind_cf_df  = wind_cf_df.resample(TIME_RES).mean()


print(pv_cf_df)


region_5             Kärnten/Steiermark  Niederösterreich/Burgenland  \
time                                                                   
2019-01-01 00:00:00            0.000000                     0.000000   
2019-01-01 03:00:00            0.000000                     0.000000   
2019-01-01 06:00:00            0.026133                     0.004093   
2019-01-01 09:00:00            0.296385                     0.083962   
2019-01-01 12:00:00            0.369237                     0.162257   
...                                 ...                          ...   
2019-12-31 09:00:00            0.428743                     0.456774   
2019-12-31 12:00:00            0.510923                     0.356018   
2019-12-31 15:00:00            0.066789                     0.032837   
2019-12-31 18:00:00            0.000000                     0.000000   
2019-12-31 21:00:00            0.000000                     0.000000   

region_5             Oberösterreich/Salzburg  Tirol/Vorarlberg 

In [ ]:
n = pypsa.Network()
n.set_snapshots(pv_cf_df.index)

load_mapped = load_mapped.reindex(n.snapshots).ffill()
pv_cf_df    = pv_cf_df.reindex(n.snapshots)
wind_cf_df  = wind_cf_df.reindex(n.snapshots)


In [ ]:
carrier_co2 = {"electricity":0.0, "solar":0.0, "wind":0.0, "hydro":0.0, "gas":0.0, "battery":0.0, "H2":0.0}
for c, co2 in carrier_co2.items():
    if c not in n.carriers.index:
        n.add("Carrier", c, co2_emissions=co2)


In [ ]:
#tech assum

tech_projections_df = pd.read_csv(r"C:\Users\steli\DSESM\data\pypsa_model\costs_2030.csv")

need = [
    ("solar", "investment"), ("solar", "FOM"), ("solar", "lifetime"), ("solar", "VOM"),
    ("onwind", "investment"), ("onwind", "FOM"), ("onwind", "lifetime"), ("onwind", "VOM"),
    ("gas", "fuel"), ("gas", "CO2 intensity"),
    ("battery inverter", "investment"), ("battery inverter", "FOM"), ("battery inverter", "lifetime"),
    ("battery storage", "investment"), ("battery storage", "lifetime"),
    ("hydrogen storage tank type 1 including compressor", "investment"),
    ("hydrogen storage tank type 1 including compressor", "lifetime"),
]

vals = {
    (t, p): float(tech_projections_df.loc[(tech_projections_df.technology == t) & (tech_projections_df.parameter == p), "value"].iloc[0])
    for (t, p) in need
}

vals

{('solar', 'investment'): 543.3289,
 ('solar', 'FOM'): 1.9495,
 ('solar', 'lifetime'): 40.0,
 ('solar', 'VOM'): 0.0106,
 ('onwind', 'investment'): 1095.8533,
 ('onwind', 'FOM'): 1.2167,
 ('onwind', 'lifetime'): 30.0,
 ('onwind', 'VOM'): 1.4286,
 ('gas', 'fuel'): 24.568,
 ('gas', 'CO2 intensity'): 0.198,
 ('battery inverter', 'investment'): 170.144,
 ('battery inverter', 'FOM'): 0.3375,
 ('battery inverter', 'lifetime'): 10.0,
 ('battery storage', 'investment'): 151.0028,
 ('battery storage', 'lifetime'): 25.0,
 ('hydrogen storage tank type 1 including compressor', 'investment'): 47.7573,
 ('hydrogen storage tank type 1 including compressor', 'lifetime'): 30.0}

In [ ]:
# --- COSTS (manual values from costs_2030.csv / tech-data) ---

r = 0.07
KW_TO_MW = 1000.0
KWH_TO_MWH = 1000.0

# values copied manually from costs_2030.csv
V = {
    ("solar", "investment"): 543.3289,
    ("solar", "FOM"): 1.9495,
    ("solar", "lifetime"): 40.0,
    ("solar", "VOM"): 0.0106,

    ("onwind", "investment"): 1095.8533,
    ("onwind", "FOM"): 1.2167,
    ("onwind", "lifetime"): 30.0,
    ("onwind", "VOM"): 1.4286,

    ("gas", "fuel"): 24.568,                 # EUR/MWh_th
    ("gas", "CO2 intensity"): 0.198,         # tCO2/MWh_th

    ("CCGT", "efficiency"): 0.58,            # p.u.
    ("CCGT", "VOM"): 4.4445,                 # EUR/MWh_el

    ("battery inverter", "investment"): 170.144,
    ("battery inverter", "FOM"): 0.3375,
    ("battery inverter", "lifetime"): 10.0,

    ("battery storage", "investment"): 151.0028,
    ("battery storage", "lifetime"): 25.0,

    ("hydrogen storage tank type 1 including compressor", "investment"): 47.7573,
    ("hydrogen storage tank type 1 including compressor", "lifetime"): 30.0,
}

def A(n):
    return r / (1 - (1 + r) ** (-n))

def capcost_kw(inv_kw, fom_pct, life_y):
    inv_mw = inv_kw * KW_TO_MW
    return A(life_y) * inv_mw + (fom_pct / 100.0) * inv_mw   # EUR/MW/a

def capcost_kwh(inv_kwh, life_y):
    inv_mwh = inv_kwh * KWH_TO_MWH
    return A(life_y) * inv_mwh                               # EUR/MWh/a

# PV/Wind
cap_cost = {
    "solar": capcost_kw(V[("solar", "investment")], V[("solar", "FOM")], V[("solar", "lifetime")]),
    "wind":  capcost_kw(V[("onwind", "investment")], V[("onwind", "FOM")], V[("onwind", "lifetime")]),
}

marg_cost = {
    "solar": V[("solar", "VOM")],
    "wind":  V[("onwind", "VOM")],
}

# Gas (use CCGT tech for efficiency + VOM; fuel+CO2 are per MWh_th)
ETA_GAS = V[("CCGT", "efficiency")]
marg_cost["gas"] = V[("gas", "fuel")] / ETA_GAS + V[("CCGT", "VOM")]
gas_co2_el = V[("gas", "CO2 intensity")] / ETA_GAS

# Battery StorageUnits (2/4/6h): power part (inverter) + energy part (storage * max_hours)
bat_power_cost = capcost_kw(V[("battery inverter", "investment")], V[("battery inverter", "FOM")], V[("battery inverter", "lifetime")])
bat_energy_cost = capcost_kwh(V[("battery storage", "investment")], V[("battery storage", "lifetime")])

battery_cap_cost = {h: bat_power_cost + bat_energy_cost * h for h in [2, 4, 6]}
marg_cost["battery"] = 0.0

# H2 StorageUnits (168/336/672h) using bundled tank+compressor (€/kWh)
h2_energy_cost = capcost_kwh(
    V[("hydrogen storage tank type 1 including compressor", "investment")],
    V[("hydrogen storage tank type 1 including compressor", "lifetime")],
)
h2_cap_cost = {h: h2_energy_cost * h for h in [168, 336, 672]}
marg_cost["H2"] = 0.0


In [ ]:
if "gas" not in n.carriers.index:
    n.add("Carrier", "gas", co2_emissions=gas_co2_el)
else:
    n.carriers.loc["gas", "co2_emissions"] = gas_co2_el



In [ ]:
###Buses
for region, row in regions.iterrows():
    n.add("Bus", region, x=row.REP_POINT.x, y=row.REP_POINT.y, carrier="electricity")


In [ ]:
##Loads
for region in regions.index:
    n.add("Load", f"load_{region}", bus=region, p_set=load_mapped[region])



In [ ]:
gdf_pp = get_existing_conv_pp()

regions_3035 = regions.to_crs(3035)
pp_3035 = gdf_pp.to_crs(3035)

regions_join = regions_3035[["geometry"]].copy()
regions_join["region"] = regions_join.index

pp_with_region = gpd.sjoin(
    pp_3035,
    regions_join[["region","geometry"]],
    how="left",
    predicate="within"
).dropna(subset=["region"])

conv_cap = (
    pp_with_region
    .groupby(["region","primary_fuel"])["capacity_mw"]
    .sum()
    .loc[lambda s: s.index.get_level_values("primary_fuel").isin(["Gas","Hydro"])]
)

for c in ["gas","hydro"]:
    if c not in n.carriers.index:
        n.add("Carrier", c)

for (region, fuel), cap in conv_cap.items():
    if fuel == "Gas":
        n.add("Generator", f"gas_existing_{region}",
              bus=region, carrier="gas",
              p_nom=float(cap), p_nom_extendable=False,
              efficiency=0.50, marginal_cost=marg_cost["gas"])

    if fuel == "Hydro":
        n.add("Generator", f"hydro_existing_{region}",
              bus=region, carrier="hydro",
              p_nom=float(cap), p_nom_extendable=False,
              p_max_pu=0.45, marginal_cost=5.0)


In [ ]:
###Renewables

for region in regions.index:
    n.add(
        "Generator", f"PV_{region}",
        bus=region, carrier="solar",
        p_nom=0.0, p_nom_extendable=True, p_nom_max=float(pv_cap_s.loc[region]),
        p_max_pu=pv_cf_df[region],
        capital_cost=cap_cost["solar"],
        marginal_cost=marg_cost["solar"],
    )

for region in regions.index:
    n.add(
        "Generator", f"Wind_{region}",
        bus=region, carrier="wind",
        p_nom=0.0, p_nom_extendable=True, p_nom_max=float(wind_cap_s.loc[region]),
        p_max_pu=wind_cf_df[region],
        capital_cost=cap_cost["wind"],
        marginal_cost=marg_cost["wind"],
    )


In [ ]:
###Transmission links

neighbours = [
    ("Wien", "Niederösterreich/Burgenland"),
    ("Niederösterreich/Burgenland", "Oberösterreich/Salzburg"),
    ("Oberösterreich/Salzburg", "Tirol/Vorarlberg"),
    ("Oberösterreich/Salzburg", "Kärnten/Steiermark"),
]

rep_points = regions["REP_POINT"].to_crs(3035)
dist_km = lambda a, b: 1.5 * rep_points.loc[a].distance(rep_points.loc[b]) / 1000

COST_PER_MW_KM = 700.0

for r1, r2 in neighbours:
    n.add(
        "Link", f"line_{r1}_{r2}",
        bus0=r1, bus1=r2,
        p_nom_extendable=True,
        p_min_pu=-1, p_max_pu=1,
        efficiency=1.0,
        capital_cost=COST_PER_MW_KM * dist_km(r1, r2)
    )

In [ ]:
#Storage

for r in regions.index:
    for h in [2, 4, 6]:
        n.add(
            "StorageUnit", f"battery_{h}h_{r}",
            bus=r, carrier="battery",
            p_nom_extendable=True,
            max_hours=h,
            capital_cost=battery_cap_cost[h],
            marginal_cost=marg_cost["battery"],
            efficiency_store=0.96,
            efficiency_dispatch=0.96,
            cyclic_state_of_charge=True
        )


for r in regions.index:
    for h in [168, 336, 672]:
        n.add(
            "StorageUnit", f"H2_{h}h_{r}",
            bus=r, carrier="H2",
            p_nom_extendable=True,
            max_hours=h,
            capital_cost=h2_cap_cost[h],
            marginal_cost=marg_cost["H2"],
            efficiency_store=0.95,
            efficiency_dispatch=0.95,
            cyclic_state_of_charge=True
        )



In [ ]:
#Run 1
n1 = n.copy()
n1.optimize(solver_name="highs", log_to_console=False)

# Run 2 (CO₂ = 0)
#n2 = n.copy()
#n2.add("GlobalConstraint", "CO2Limit", type="primary_energy", carrier_attribute="co2_emissions", sense="<=", constant=0.0)
#n2.optimize(solver_name="highs", log_to_console=False)

# — Quick outputs 
cap_gen_1 = n1.generators.p_nom_opt.groupby(n1.generators.carrier).sum()
#cap_gen_2 = n2.generators.p_nom_opt.groupby(n2.generators.carrier).sum()

mix1 = n1.generators_t.p.sum().groupby(n1.generators.carrier).sum()
#mix2 = n2.generators_t.p.sum().groupby(n2.generators.carrier).sum()

#cap_gen_1, cap_gen_2, mix1, mix2, n1.objective, n2.objective

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 8/8 [00:00<00:00, 13.53it/s]
INFO:linopy.io: Writing time: 4.59s
